# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 15 2022 1:08PM,250828,10,PRF-40692,Bio-PRF,Released
1,Nov 15 2022 1:08PM,250828,10,SMT-1149,Bio-PRF,Released
2,Nov 15 2022 1:08PM,250828,10,PRF-41891,Bio-PRF,Released
3,Nov 15 2022 1:08PM,250828,10,PRF-41895,Bio-PRF,Released
4,Nov 15 2022 1:08PM,250828,10,PRF-41898,Bio-PRF,Released
5,Nov 15 2022 1:08PM,250828,10,PRF-41903,Bio-PRF,Released
6,Nov 15 2022 1:08PM,250828,10,8779229,Bio-PRF,Released
7,Nov 15 2022 1:08PM,250828,10,PRF-41986,Bio-PRF,Released
8,Nov 15 2022 1:08PM,250828,10,PRF-41562,Bio-PRF,Released
9,Nov 15 2022 1:02PM,250825,10,MTR-22-0270.2,"Senseonics, Incorporated",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,250822,Released,1
40,250823,Released,1
41,250824,Released,1
42,250825,Released,1
43,250828,Released,9


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
250822,NaN,NaN,1.0
250823,NaN,NaN,1.0
250824,NaN,NaN,1.0
250825,NaN,NaN,1.0
250828,NaN,NaN,9.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250670,0.0,0.0,48.0
250755,0.0,14.0,15.0
250757,0.0,0.0,1.0
250766,0.0,0.0,1.0
250768,0.0,18.0,41.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250670,0,0,48
250755,0,14,15
250757,0,0,1
250766,0,0,1
250768,0,18,41


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250670,0,0,48
1,250755,0,14,15
2,250757,0,0,1
3,250766,0,0,1
4,250768,0,18,41


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250670,,,48
1,250755,,14,15
2,250757,,,1
3,250766,,,1
4,250768,,18,41


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 15 2022 1:08PM,250828,10,Bio-PRF
9,Nov 15 2022 1:02PM,250825,10,"Senseonics, Incorporated"
10,Nov 15 2022 1:02PM,250824,10,"Nextsource Biotechnology, LLC"
11,Nov 15 2022 1:01PM,250823,10,"Senseonics, Incorporated"
12,Nov 15 2022 1:01PM,250821,12,Hush Hush
20,Nov 15 2022 12:59PM,250822,21,"NBTY Global, Inc."
21,Nov 15 2022 12:58PM,250819,12,"Uniderm USA, Inc"
23,Nov 15 2022 12:54PM,250818,10,"Virtue Rx, LLC"
24,Nov 15 2022 12:50PM,250816,19,"AdvaGen Pharma, Ltd"
28,Nov 15 2022 12:50PM,250817,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 15 2022 1:08PM,250828,10,Bio-PRF,,,9
1,Nov 15 2022 1:02PM,250825,10,"Senseonics, Incorporated",,,1
2,Nov 15 2022 1:02PM,250824,10,"Nextsource Biotechnology, LLC",,,1
3,Nov 15 2022 1:01PM,250823,10,"Senseonics, Incorporated",,,1
4,Nov 15 2022 1:01PM,250821,12,Hush Hush,,,8
5,Nov 15 2022 12:59PM,250822,21,"NBTY Global, Inc.",,,1
6,Nov 15 2022 12:58PM,250819,12,"Uniderm USA, Inc",,,2
7,Nov 15 2022 12:54PM,250818,10,"Virtue Rx, LLC",,,1
8,Nov 15 2022 12:50PM,250816,19,"AdvaGen Pharma, Ltd",,2,2
9,Nov 15 2022 12:50PM,250817,10,Emerginnova,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 1:08PM,250828,10,Bio-PRF,9,,
1,Nov 15 2022 1:02PM,250825,10,"Senseonics, Incorporated",1,,
2,Nov 15 2022 1:02PM,250824,10,"Nextsource Biotechnology, LLC",1,,
3,Nov 15 2022 1:01PM,250823,10,"Senseonics, Incorporated",1,,
4,Nov 15 2022 1:01PM,250821,12,Hush Hush,8,,
5,Nov 15 2022 12:59PM,250822,21,"NBTY Global, Inc.",1,,
6,Nov 15 2022 12:58PM,250819,12,"Uniderm USA, Inc",2,,
7,Nov 15 2022 12:54PM,250818,10,"Virtue Rx, LLC",1,,
8,Nov 15 2022 12:50PM,250816,19,"AdvaGen Pharma, Ltd",2,2,
9,Nov 15 2022 12:50PM,250817,10,Emerginnova,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 1:08PM,250828,10,Bio-PRF,9,,
1,Nov 15 2022 1:02PM,250825,10,"Senseonics, Incorporated",1,,
2,Nov 15 2022 1:02PM,250824,10,"Nextsource Biotechnology, LLC",1,,
3,Nov 15 2022 1:01PM,250823,10,"Senseonics, Incorporated",1,,
4,Nov 15 2022 1:01PM,250821,12,Hush Hush,8,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 1:08PM,250828,10,Bio-PRF,9.0,NaN,NaN
1,Nov 15 2022 1:02PM,250825,10,"Senseonics, Incorporated",1.0,NaN,NaN
2,Nov 15 2022 1:02PM,250824,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
3,Nov 15 2022 1:01PM,250823,10,"Senseonics, Incorporated",1.0,NaN,NaN
4,Nov 15 2022 1:01PM,250821,12,Hush Hush,8.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 1:08PM,250828,10,Bio-PRF,9.0,0.0,0.0
1,Nov 15 2022 1:02PM,250825,10,"Senseonics, Incorporated",1.0,0.0,0.0
2,Nov 15 2022 1:02PM,250824,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
3,Nov 15 2022 1:01PM,250823,10,"Senseonics, Incorporated",1.0,0.0,0.0
4,Nov 15 2022 1:01PM,250821,12,Hush Hush,8.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5266926,239.0,19.0,0.0
12,1003213,12.0,0.0,0.0
15,752253,92.0,18.0,0.0
16,250766,1.0,0.0,0.0
19,752421,2.0,4.0,0.0
20,752386,18.0,0.0,4.0
21,1253974,5.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5266926,239.0,19.0,0.0
1,12,1003213,12.0,0.0,0.0
2,15,752253,92.0,18.0,0.0
3,16,250766,1.0,0.0,0.0
4,19,752421,2.0,4.0,0.0
5,20,752386,18.0,0.0,4.0
6,21,1253974,5.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,239.0,19.0,0.0
1,12,12.0,0.0,0.0
2,15,92.0,18.0,0.0
3,16,1.0,0.0,0.0
4,19,2.0,4.0,0.0
5,20,18.0,0.0,4.0
6,21,5.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,239.0
1,12,Released,12.0
2,15,Released,92.0
3,16,Released,1.0
4,19,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,4.0,0.0
Executing,19.0,0.0,18.0,0.0,4.0,0.0,0.0
Released,239.0,12.0,92.0,1.0,2.0,18.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,4.0,0.0
1,Executing,19.0,0.0,18.0,0.0,4.0,0.0,0.0
2,Released,239.0,12.0,92.0,1.0,2.0,18.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,4.0,0.0
1,Executing,19.0,0.0,18.0,0.0,4.0,0.0,0.0
2,Released,239.0,12.0,92.0,1.0,2.0,18.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()